<a href="https://colab.research.google.com/github/CesarPortales/BootCamp/blob/master/HADOOP_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HADOOP desde CERO

## Intro

No es más que una colección de herramientas open source (algunas hasta closed source).

![ML xkcd](https://imgs.xkcd.com/comics/machine_learning.png)

## Primero instalemos y probemos HADOOP 2.7.+


```
docker ps

docker run hello-world

docker pull sequenceiq/hadoop-docker:2.7.1

```
### Hasta aqui que ha pasado?

[A la pizarra!](https://witeboard.com/9734d460-7306-11e8-904a-07457219cdcb)  

[Mejor en G Draw](https://docs.google.com/drawings/d/1-OmMtvMg1STC2RIMlmL4gCxi6MB_3evlwsOhn4RgeVU)

http://merrigrove.blogspot.com/2015/10/visualizing-docker-containers-and-images.html

### Entremos en accion

```
docker run -p 50070:50070 -p 8088:8088 -it  sequenceiq/hadoop-docker:2.7.1 /bin/bash -bash

/etc/bootstrap.sh

ls

pwd

cat

tail

mkdir

```


### Es HADOOP UNIX sin mas?

Hablemos de POSIX:

- **P**ortable **O**perating **S**ystem **I**nterface (UNI**X**)
- Creado por Richard Stallman
- *PORTABLE*
- HDFS no es POSIX

![posix](https://fossbytes.com/wp-content/uploads/2016/09/posix-ieee.jpg)


https://www.slideshare.net/mcsrivas/design-scale-and-performance-of-maprs-distribution-for-hadoop

**RAZON PRINCIPAL**: "A key one is that you can only add data to the end of the file, not seek to the middle and write things. You cannot seamlessly map code that assumes that all filesystems are Posix-compatible to HDFS."
  
    

## HDFS

![data store](http://csis.pace.edu/~marchese/CS865/Lectures/Chap7/Chapter7fin_files/image002.jpg)

### Que debe existir dentro de un HDFS?

- Ficheros inmutables (csv, txt, etc)
- Jars de mapreduce
- Librerias
- Ficheros de secuencia

### Juguemos con HDFS

```
cd $HADOOP_PREFIX

bin/hdfs dfs -mkdir pruebas

bin/hdfs dfs -put README.txt pruebas/README.txt # Va a fallar si no existe la carpeta en hdfs

bin/hdfs dfs -ls pruebas

bin/hdfs dfs -touchz pruebas/test.zip

bin/hdfs dfs -ls pruebas

bin/hdfs dfs -cat pruebas/*

bin/hdfs dfs

jps
```


## HIGH AVAILABILITY
- FAULT TOLERANCE
- Resilient
- Elasticity


 ### JPS???
 
 ![jps arch](https://hadoop.apache.org/docs/r1.2.1/images/hdfsarchitecture.gif)

* HDFS tiene una arquitectura maestro/esclavo. 
* Un clúster HDFS consta de un único NameNode, un servidor maestro que administra el espacio de nombres del sistema de archivos y regula el acceso a los archivos por parte de los clientes. 
* Los DataNodes administran el almacenamiento.

### Arquitectura HDFS

* Los NameNode y DataNode son piezas de software diseñadas para ejecutarse en máquinas básicas. 
* Estas máquinas suelen ejecutar un sistema operativo GNU / Linux. 
* HDFS está construido usando en Java.
* La existencia de un NameNode único en un clúster simplifica enormemente la arquitectura del sistema. 
* NameNode es el árbitro y repositorio de todos los metadatos HDFS. 
* El sistema está diseñado de tal manera que los datos del usuario nunca fluyan a través del NameNode.
* La ubicacion fisica de los nodos es de vital importancia y requiere mucha practica (ensayo y error).

### Mas lectura:

- https://hadoop.apache.org/docs/r1.2.1/hdfs_design.html
- https://fossbytes.com/posix-what-is-the-portable-operating-system-interface/


## MAP REDUCE

### Juguemos con Map Reduce

```
cd $HADOOP_PREFIX

bin/hadoop jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar grep input output 'dfs[a-z.]+'

```

### Pero que ha pasado?

Busquemos la IP del cluster, desde nuestro host:

```
docker network list

docker network inspect docker_default

```
Fijarse en gateway, y cambiar el `1` por un `2`.

Visitemos: http://127.0.0.1:50070 * NOTA - WINDOWS - En docker ANTES de hacer el run aparece el IP en verde, de esta manera:
![winip](https://i.imgur.com/Z8g9n4d.png)


### Que otra forma tendriamos de ver los resultados?


### Un poco de Teoria obligatoria

map: reading and formatting data
reduce: applying transformations/operations on all of the data

![mr](https://www.todaysoftmag.com/images/articles/tsm33/large/a11.png)


### Veamos el Codigo

```java
import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class WordCount {

  public static class TokenizerMapper
       extends Mapper<Object, Text, Text, IntWritable>{

    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(Object key, Text value, Context context
                    ) throws IOException, InterruptedException {
      StringTokenizer itr = new StringTokenizer(value.toString());
      while (itr.hasMoreTokens()) {
        word.set(itr.nextToken());
        context.write(word, one);
      }
    }
  }

  public static class IntSumReducer
       extends Reducer<Text,IntWritable,Text,IntWritable> {
    private IntWritable result = new IntWritable();

    public void reduce(Text key, Iterable<IntWritable> values,
                       Context context
                       ) throws IOException, InterruptedException {
      int sum = 0;
      for (IntWritable val : values) {
        sum += val.get();
      }
      result.set(sum);
      context.write(key, result);
    }
  }

  public static void main(String[] args) throws Exception {
    Configuration conf = new Configuration();
    Job job = Job.getInstance(conf, "word count");
    job.setJarByClass(WordCount.class);
    job.setMapperClass(TokenizerMapper.class);
    job.setCombinerClass(IntSumReducer.class);
    job.setReducerClass(IntSumReducer.class);
    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);
    FileInputFormat.addInputPath(job, new Path(args[0]));
    FileOutputFormat.setOutputPath(job, new Path(args[1]));
    System.exit(job.waitForCompletion(true) ? 0 : 1);
  }
}

```


### Mas ejemplos

```
bin/hadoop jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar
```

### Mas lectura

- https://www.todaysoftmag.com/article/1358/hadoop-mapreduce-deep-diving-and-tuning